In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing Libraries

In [63]:
import numpy as np
import copy
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from itertools import product
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Loading Dataset

In [64]:
import pandas as pd
df = pd.read_excel(r'/content/drive/MyDrive/AI Hackathon /Convolve /Round 2/Copy of FileAndResults_Problem2.xlsx')

In [65]:
# Interaction Features
df['Coder_Tester_Interact'] = df['Coder'] + "_" + df['Tester']
df['Category_Component_Interact'] = df['Category'] + "_" + df['Component']

In [66]:
df.head(5)

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result,Coder_Tester_Interact,Category_Component_Interact
0,cpp/csyu/category/fnf,asr1001-flowpsk,Batukbaj,samsk,PD-asr1001,Thor_Vista,NCS5500,Pass,Batukbaj_samsk,PD-asr1001_asr1001-flowpsk
1,ignor-dp/build,asr1001-ignor,Alex,manishakang,Tools,vistra-cxr,NCS5500,Aborted,Alex_manishakang,Tools_asr1001-ignor
2,ignor-dp/cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Fail,Alex_manishakang,PD-asr1001_asr1001-ignor
3,ignor-dp/csco/affinity,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted,Alex_manishakang,PD-asr1001_asr1001-ignor
4,ignor-dp/csco/cdp,asr1001-ignor,Alex,manishakang,PD-asr1001,vistra-cxr,NCS5500,Aborted,Alex_manishakang,PD-asr1001_asr1001-ignor


In [67]:
df.describe()

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result,Coder_Tester_Interact,Category_Component_Interact
count,351,351,351,337,351,351,351,351,337,351
unique,176,45,7,18,11,25,14,3,29,73
top,cpp/csyu/category/fnf,asr1001-forge-cgn,patrick,manishakang,PD-asr1001,vistra-cxr,"asr1001-PX,asr1001-X64",Pass,patrick_manishakang,PD-asr1001_asr1001-forge-cgn
freq,2,70,130,109,146,122,105,210,95,66


In [68]:
df.shape

(351, 10)

#Data Pre-processing



In [69]:
#Removing "/" from "File changed" i.e changing "ignor-dp/csco/logrotate" to "ignor-dp csco logrotate".
dir = list(df['File Changed'])
new_dir = []
for i in dir:
  j = i.replace("/"," ")
  new_dir.append(j)
df['File Changed'] = new_dir

In [70]:
df.head(2)

,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result,Coder_Tester_Interact,Category_Component_Interact
0,cpp csyu category fnf,asr1001-flowpsk,Batukbaj,samsk,PD-asr1001,Thor_Vista,NCS5500,Pass,Batukbaj_samsk,PD-asr1001_asr1001-flowpsk
1,ignor-dp build,asr1001-ignor,Alex,manishakang,Tools,vistra-cxr,NCS5500,Aborted,Alex_manishakang,Tools_asr1001-ignor


In [71]:
headers=['File Changed','Component','Coder','Tester','Category','Sanity name','nets Tested','Sanity Result','Coder_Tester_Interact','Category_Component_Interact']
count=[176,45,7,18,11,25,14,3,29,73]

# Loop over the specified columns for categorical data transformation
for i in range(1,10):
        df[headers[i]]=df[headers[i]].astype('category')
        df[headers[i]]=df[headers[i]].cat.codes

In [72]:
df.head()


,File Changed,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result,Coder_Tester_Interact,Category_Component_Interact
0,cpp csyu category fnf,5,1,12,4,8,1,2,8,23
1,ignor-dp build,7,0,7,9,17,1,0,0,70
2,ignor-dp cdp,7,0,7,4,17,1,1,0,25
3,ignor-dp csco affinity,7,0,7,4,17,1,0,0,25
4,ignor-dp csco cdp,7,0,7,4,17,1,0,0,25


In [73]:
# Function for Bag of Words
def bag_of_word(name,df,a,b):
  new_dir1=df[name]
  cv = CountVectorizer()
  X = cv.fit_transform(new_dir1)
  sorted(cv.vocabulary_.keys())
  n=len(cv.vocabulary_.keys())
  X = X.toarray()
  f = list(range(a,b))
  f = map(str,f)
  data = pd.DataFrame(X, columns=f )
  return data,n

In [74]:
df['Sanity Result'].value_counts()

2    210
1     71
0     70
Name: Sanity Result, dtype: int64

In [75]:
from sklearn.utils.class_weight import compute_class_weight

y = df['Sanity Result']

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)

# Print the class weights
print("Class Weights:", class_weights)
class_weights={0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}

Class Weights: [1.67142857 1.64788732 0.55714286]


In [33]:
# Calculate the total number of samples just as the above code
total_samples = len(y)

# Count the number of samples for each class
class_counts = {}
for label in y:
    class_counts[label] = class_counts.get(label, 0) + 1

# Calculate class weights
class_weights = {}
for label, count in class_counts.items():
    class_weights[label] = total_samples / (len(class_counts) * count)

# Print the class weights
print("Class Weights:", class_weights)


Class Weights: {2: 0.5571428571428572, 0: 1.6714285714285715, 1: 1.647887323943662}


In [76]:
#Changing "File Change",using Bag of words.
data1,n=bag_of_word('File Changed',df,1,137)
frames = [data1,df]
result = pd.concat(frames, axis=1, join='inner')
df=result.drop(['File Changed'],axis=1)
df.head(5)

,1,2,3,4,5,6,7,8,9,10,...,136,Component,Coder,Tester,Category,Sanity name,nets Tested,Sanity Result,Coder_Tester_Interact,Category_Component_Interact
0,0,0,0,0,0,0,0,0,0,0,...,0,5,1,12,4,8,1,2,8,23
1,0,0,0,0,0,0,0,0,0,0,...,0,7,0,7,9,17,1,0,0,70
2,0,0,0,0,0,0,0,0,0,0,...,0,7,0,7,4,17,1,1,0,25
3,0,1,0,0,0,0,0,0,0,0,...,0,7,0,7,4,17,1,0,0,25
4,0,0,0,0,0,0,0,0,0,0,...,0,7,0,7,4,17,1,0,0,25


In [77]:
y=df['Sanity Result']
X=df.drop(['Sanity Result'],axis=1)

In [78]:
X.head()

,1,2,3,4,5,6,7,8,9,10,...,135,136,Component,Coder,Tester,Category,Sanity name,nets Tested,Coder_Tester_Interact,Category_Component_Interact
0,0,0,0,0,0,0,0,0,0,0,...,0,0,5,1,12,4,8,1,8,23
1,0,0,0,0,0,0,0,0,0,0,...,0,0,7,0,7,9,17,1,0,70
2,0,0,0,0,0,0,0,0,0,0,...,0,0,7,0,7,4,17,1,0,25
3,0,1,0,0,0,0,0,0,0,0,...,0,0,7,0,7,4,17,1,0,25
4,0,0,0,0,0,0,0,0,0,0,...,0,0,7,0,7,4,17,1,0,25


In [79]:
X.shape

(351, 144)

#Splitting Dataset

In [80]:
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA

# Feature Selection using Random Forest Feature Importance
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X, y)
feature_importances = forest.feature_importances_
feature_names = X.columns

# Select features with importance greater than a threshold
threshold = 0.001
selected_features_rf = feature_names[feature_importances > threshold]

# Print the number of selected features
print("Number of selected features (Random Forest):", len(selected_features_rf))


Number of selected features (Random Forest): 108


In [81]:
print(len(selected_features_rf))
X=X[selected_features_rf]

108


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
X_train.shape

(280, 108)

#Training Phase

In [84]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif

# Perform hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestClassifier(random_state=42,class_weight=class_weights)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Get the best hyperparameters
best_params_rf = grid_search_rf.best_params_
print(best_params_rf)

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [85]:
#Random Forest
from sklearn.model_selection import cross_val_score
forest = RandomForestClassifier(**best_params_rf,class_weight=class_weights)

forest = forest.fit(X_train,y_train)
r=forest.predict(X_test)
probab = forest.predict_proba(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
print(cm)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(metrics.classification_report(y_test, r))
cv_scores = cross_val_score(forest, X_train, y_train, cv=5)  # Adjust cv as needed


# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = forest.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = forest.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

94.36619718309859
[[14  0  2]
 [ 0 11  1]
 [ 1  0 42]]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        16
           1       1.00      0.92      0.96        12
           2       0.93      0.98      0.95        43

    accuracy                           0.94        71
   macro avg       0.96      0.92      0.94        71
weighted avg       0.94      0.94      0.94        71

Bias: 0.16785714285714293
Variance: 0.051259643194311866
Training Accuracy: 1.0
Test Accuracy: 0.9436619718309859


In [86]:
#LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='newton-cg',class_weight=class_weights)
model.fit(X_train, y_train)
r=model.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
print(cm)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(metrics.classification_report(y_test, r))
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

76.05633802816901
[[16  0  0]
 [ 0 10  2]
 [ 7  8 28]]
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        16
           1       0.56      0.83      0.67        12
           2       0.93      0.65      0.77        43

    accuracy                           0.76        71
   macro avg       0.73      0.83      0.75        71
weighted avg       0.82      0.76      0.76        71

Bias: 0.3571428571428571
Variance: 0.0852666884736666
Training Accuracy: 0.8392857142857143
Test Accuracy: 0.7605633802816901


In [87]:
#DecisionTree
dtree = DecisionTreeClassifier(class_weight=class_weights)
dtree = dtree.fit(X_train, y_train)
r=dtree.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test, r)*100)
print(cm)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(metrics.classification_report(y_test, r))

# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = dtree.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = dtree.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

87.32394366197182
[[12  3  1]
 [ 0 11  1]
 [ 2  2 39]]
              precision    recall  f1-score   support

           0       0.86      0.75      0.80        16
           1       0.69      0.92      0.79        12
           2       0.95      0.91      0.93        43

    accuracy                           0.87        71
   macro avg       0.83      0.86      0.84        71
weighted avg       0.89      0.87      0.88        71

Bias: 0.3571428571428571
Variance: 0.0852666884736666
Training Accuracy: 1.0
Test Accuracy: 0.8732394366197183


In [88]:
#GaussianNB
gnb = GaussianNB()
a = gnb.fit(X_train, y_train)
y_pred = a.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
print(metrics.accuracy_score(y_test, y_pred)*100)
print(cm)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(metrics.classification_report(y_test, r))

# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = gnb.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = gnb.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

56.33802816901409
[[16  0  0]
 [ 7  4  1]
 [14  9 20]]
              precision    recall  f1-score   support

           0       0.86      0.75      0.80        16
           1       0.69      0.92      0.79        12
           2       0.95      0.91      0.93        43

    accuracy                           0.87        71
   macro avg       0.83      0.86      0.84        71
weighted avg       0.89      0.87      0.88        71

Bias: 0.3571428571428571
Variance: 0.0852666884736666
Training Accuracy: 0.5642857142857143
Test Accuracy: 0.5633802816901409


In [89]:

#XGBoostClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
print(metrics.accuracy_score(y_test, y_pred)*100)
print(cm)
# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

90.14084507042254
[[15  0  1]
 [ 1  9  2]
 [ 2  1 40]]
Bias: 0.3571428571428571
Variance: 0.0852666884736666
Training Accuracy: 0.9785714285714285
Test Accuracy: 0.9014084507042254


In [90]:
#Ensemble model
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
m1 = LogisticRegression(multi_class='multinomial', solver='newton-cg',class_weight=class_weights)
m2 = RandomForestClassifier(n_estimators = 50,class_weight=class_weights)
eclf1 = VotingClassifier(estimators=[('l1', m1), ('l2',m2)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
r = eclf1.predict(X_test)
cm = confusion_matrix(y_test,r)
print(metrics.accuracy_score(y_test,r)*100)
print(cm)
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(metrics.classification_report(y_test, r))
# Calculate bias and variance
mean_cv_score = cv_scores.mean()
bias = 1 - mean_cv_score
variance = cv_scores.std()

print("Bias:", bias)
print("Variance:", variance)
# Make predictions on training set
y_train_pred = eclf1.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
y_test_pred = eclf1.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

95.77464788732394
[[16  0  0]
 [ 0 11  1]
 [ 2  0 41]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.92      0.96        12
           2       0.98      0.95      0.96        43

    accuracy                           0.96        71
   macro avg       0.96      0.96      0.95        71
weighted avg       0.96      0.96      0.96        71

Bias: 0.3571428571428571
Variance: 0.0852666884736666
Training Accuracy: 0.9964285714285714
Test Accuracy: 0.9577464788732394


In [91]:
#Finding probabilty of "Fail","Pass" and "Aborted"
l1=[]
l2=[]
l3=[]
for i in probab:
  l1.append(i[0])
  l2.append(i[1])
  l3.append(i[2])
X_test['Prob_Aborted']=l1
X_test['Prob_Fail']=l2
X_test['Prob_Pass']=l3
# X_test['Predicted']=r
l4=[]
for i in r:
  if(i==0):
    l4.append('Aborted')
  elif(i==1):
    l4.append('Fail')
  else:
    l4.append('Pass')
X_test['Predicted']=l4
X_test

,1,2,4,5,6,7,8,10,11,12,...,Tester,Category,Sanity name,nets Tested,Coder_Tester_Interact,Category_Component_Interact,Prob_Aborted,Prob_Fail,Prob_Pass,Predicted
157,0,0,0,0,0,0,0,0,0,0,...,7,4,17,10,25,24,0.64,0.08,0.28,Aborted
342,0,0,0,0,0,0,0,0,0,0,...,7,4,17,10,0,24,0.08,0.43,0.49,Pass
316,0,0,0,0,0,0,0,0,0,0,...,7,4,17,10,25,24,0.06,0.17,0.77,Pass
234,0,0,0,0,0,0,0,0,0,0,...,1,6,10,2,20,59,0.05,0.06,0.89,Pass
155,0,0,0,0,0,0,0,0,0,0,...,7,4,20,12,25,25,0.04,0.12,0.84,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0,0,0,0,0,0,0,0,0,0,...,7,10,17,1,25,72,0.32,0.10,0.58,Aborted
179,0,0,0,0,0,0,0,0,0,0,...,7,4,17,11,25,25,0.50,0.10,0.40,Aborted
199,0,0,0,0,0,0,0,0,0,0,...,6,2,0,10,13,12,0.00,0.05,0.95,Pass
327,0,0,0,0,0,0,0,0,0,0,...,7,4,17,10,25,24,0.01,0.02,0.97,Pass
